In [ ]:
# Imports principales
import os
import random
from pathlib import Path
from typing import List, Tuple

import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF

# Optuna se usará en la celda de AutoML; aquí no lo importamos aún para evitar fallos si no está instalado.


In [ ]:
# Configuración global y rutas
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

DATASET_ROOT = Path("Reconocimiento de Caracteres/datasets/synthetic")
BATCH_SIZE = 16
VAL_FRAC = 0.3
NUM_WORKERS = 4
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Device:', DEVICE)

In [ ]:
# Definición del Dataset (compatible con las carpetas images/, masks/, masks_ignore/)
class SyntheticGlyphsDataset(Dataset):
    def __init__(self, root: Path, files: List[str]):
        self.root = Path(root)
        self.images_dir = self.root / 'images'
        self.masks_dir = self.root / 'masks'
        self.masks_ignore_dir = self.root / 'masks_ignore'
        self.files = list(files)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx: int):
        fname = self.files[idx]
        img = Image.open(self.images_dir / fname).convert('RGB')
        mask = Image.open(self.masks_dir / fname).convert('L')
        ignore_path = self.masks_ignore_dir / fname
        if ignore_path.exists():
            ignore = Image.open(ignore_path).convert('L')
        else:
            ignore = Image.new('L', mask.size, 0)

        img_t = TF.to_tensor(img)
        mask_t = TF.to_tensor(mask)
        ignore_t = TF.to_tensor(ignore)

        mask_bin = (mask_t > 0.0).float()
        ignore_bin = (ignore_t > 0.0).float()

        return {'image': img_t, 'mask': mask_bin, 'ignore': ignore_bin, 'file': fname}

def build_file_list(root: Path) -> List[str]:
    images_dir = root / 'images'
    if not images_dir.exists():
        return []
    return [p.name for p in sorted(images_dir.glob('*.png'))]

def split_files(files: List[str], val_frac: float = 0.1, seed: int | None = 42) -> Tuple[List[str], List[str]]:
    rng = random.Random(seed)
    files_shuffled = list(files)
    rng.shuffle(files_shuffled)
    n = len(files_shuffled)
    n_val = int(n * val_frac)
    val = files_shuffled[:n_val]
    train = files_shuffled[n_val:]
    return train, val


In [ ]:
# Construcción de DataLoaders (train / val)
files = build_file_list(DATASET_ROOT)
print('Found files:', len(files))
train_files, val_files = split_files(files, val_frac=VAL_FRAC, seed=SEED)
print(f'Train: {len(train_files)} files, Val: {len(val_files)} files')

train_ds = SyntheticGlyphsDataset(DATASET_ROOT, train_files)
val_ds = SyntheticGlyphsDataset(DATASET_ROOT, val_files)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print('Train batches:', len(train_loader), 'Val batches:', len(val_loader))
# show example shapes if dataset non-empty
if len(train_ds) > 0:
    sample = train_ds[0]
    print('Sample keys:', list(sample.keys()))
    print('image shape', sample['image'].shape, 'mask shape', sample['mask'].shape)


## AutoML / HPO — decisión metodológica

Seleccionaremos Optuna con un pruner tipo ASHA (Successive Halving) porque permite detener trials de bajo rendimiento tempranamente y es sencillo de integrar.
Se trabajará en dos fases: búsqueda proxy (imágenes 128×128, 5 epochs por trial) y reevaluación de los mejores candidatos en mayor fidelidad.
La métrica objetivo será mIoU (Dice/IoU) calculada ignorando píxeles en `masks_ignore`.


In [ ]:
# Implementación completa de AutoML / HPO con Optuna + ASHA (incluyendo selección de arquitectura)
# Instalar optuna si no está disponible: pip install optuna
# Para visualizaciones: pip install plotly (si no está instalado)

import time
import math
try:
    import optuna
    from optuna.pruners import SuccessiveHalvingPruner
    import optuna.visualization as vis
except Exception as e:
    print('Optuna no disponible. Instalar con `pip install optuna plotly`.')
    raise

# Definimos modelos dentro del notebook para pruebas rápidas
import torch.nn as nn

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.net(x)

class SimpleUNet(nn.Module):
    def __init__(self, in_ch=3, base=32):
        super().__init__()
        self.enc1 = DoubleConv(in_ch, base)
        self.enc2 = DoubleConv(base, base*2)
        self.pool = nn.MaxPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='nearest')
        self.dec1 = DoubleConv(base*2, base)
        self.outc = nn.Conv2d(base, 1, 1)
    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        d = self.up(e2)
        d = self.dec1(d)
        return torch.sigmoid(self.outc(d))

class SimpleFCN(nn.Module):
    def __init__(self, in_ch=3, base=32):
        super().__init__()
        self.conv1 = DoubleConv(in_ch, base)
        self.conv2 = DoubleConv(base, base*2)
        self.conv3 = DoubleConv(base*2, base*4)
        self.pool = nn.MaxPool2d(2)
        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.dec1 = DoubleConv(base*4, base*2)
        self.dec2 = DoubleConv(base*2, base)
        self.outc = nn.Conv2d(base, 1, 1)
    def forward(self, x):
        c1 = self.conv1(x)
        p1 = self.pool(c1)
        c2 = self.conv2(p1)
        p2 = self.pool(c2)
        c3 = self.conv3(p2)
        u1 = self.up1(c3)
        d1 = self.dec1(u1)
        u2 = self.up2(d1)
        d2 = self.dec2(u2)
        return torch.sigmoid(self.outc(d2))

# Métrica: IoU ignorando píxeles 'ignore'
def iou_metric(pred, target, ignore_mask=None, eps=1e-7):
    # pred, target: tensors (B,1,H,W)
    pred_bin = (pred > 0.5).float()
    target = target.float()
    if ignore_mask is not None:
        mask = (ignore_mask < 0.5).float()
        pred_bin = pred_bin * mask
        target = target * mask
    inter = (pred_bin * target).sum(dim=[1,2,3])
    union = (pred_bin + target - pred_bin * target).sum(dim=[1,2,3])
    iou = ((inter + eps) / (union + eps)).mean().item()
    return iou

# Objective function for Optuna (ahora incluye selección de arquitectura)
def objective(trial: optuna.trial.Trial) -> float:
    # Seleccionar arquitectura
    model_type = trial.suggest_categorical('model_type', ['unet', 'fcn'])
    
    # Hiperparámetros comunes
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    base = trial.suggest_categorical('base', [16, 32, 48])
    weight_decay = trial.suggest_float('wd', 0.0, 1e-4, log=True)
    
    # Construir modelo según tipo
    if model_type == 'unet':
        model = SimpleUNet(in_ch=3, base=base).to(DEVICE)
    elif model_type == 'fcn':
        model = SimpleFCN(in_ch=3, base=base).to(DEVICE)
    
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = nn.BCELoss()

    # Quick train loop (proxy): 5 epochs
    n_epochs = 5
    for epoch in range(n_epochs):
        model.train()
        for batch in train_loader:
            imgs = batch['image'].to(DEVICE)
            masks = batch['mask'].to(DEVICE)
            ignores = batch['ignore'].to(DEVICE)
            pred = model(imgs)
            loss = loss_fn(pred, masks)
            opt.zero_grad()
            loss.backward()
            opt.step()
        # Validation metric
        model.eval()
        with torch.no_grad():
            ious = []
            for vb in val_loader:
                imgs = vb['image'].to(DEVICE)
                masks = vb['mask'].to(DEVICE)
                ignores = vb['ignore'].to(DEVICE)
                pred = model(imgs)
                ious.append(iou_metric(pred, masks, ignore_mask=ignores))
            val_iou = float(np.mean(ious)) if ious else 0.0
        trial.report(val_iou, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return val_iou

# Crear estudio y ejecutar la optimización
study = optuna.create_study(direction='maximize', pruner=SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50, timeout=None)

# Resultados
print('Best trial:', study.best_trial.params)
print('Best IoU:', study.best_trial.value)

# Visualizaciones
try:
    # Historial de optimización
    fig1 = vis.plot_optimization_history(study)
    fig1.show()

    # Importancia de parámetros
    fig2 = vis.plot_param_importances(study)
    fig2.show()

    # Contorno de parámetros (si hay dos parámetros numéricos)
    fig3 = vis.plot_contour(study)
    fig3.show()

    # Slice plot para parámetros
    fig4 = vis.plot_slice(study)
    fig4.show()

except Exception as e:
    print('Error en visualizaciones:', e)
    print('Asegúrate de tener plotly instalado: pip install plotly')

print('Optuna HPO completado con selección de arquitectura y visualizaciones.')

---
Siguientes pasos sugeridos:
- Ejecutar la búsqueda con `n_trials=50` y `n_epochs=5` (proxy) usando Optuna/ASHA.
- Re-entrenar los mejores 3 candidatos con mayor resolución y más epochs.
- Opcional: registrar en Weights & Biases para trazabilidad.